<a href="https://colab.research.google.com/github/Gokul0Krishna/exp/blob/main/exp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# !pip install torch torchvision pytorch-lightning pycocotools
!git clone https://github.com/facebookresearch/detr.git
%cd detr

Cloning into 'detr'...
remote: Enumerating objects: 265, done.
remote: Total 265 (delta 0), reused 0 (delta 0), pack-reused 265 (from 1)
Receiving objects: 100% (265/265), 21.19 MiB | 24.85 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/content/detr/detr


In [1]:
!unzip "/content/football players.v2i.coco.zip"

Archive:  /content/football players.v2i.coco.zip
   creating: football players.v2i.coco/
  inflating: football players.v2i.coco/README.dataset.txt  
  inflating: football players.v2i.coco/README.roboflow.txt  
   creating: football players.v2i.coco/test/
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0009_jpg.rf.8c94b6f7e2f93d77bf33fa4e740b90e0.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0011_jpg.rf.dd8cce210aef4b83e11dd8e851add934.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0012_jpg.rf.b5165914840af0159721c1b774c0f743.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0024_jpg.rf.6032a83254029bf74e2710a86f9f36cb.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0034_jpg.rf.cdfec2540651bc946b57fc5b9f05dcf9.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0038_jpg.rf.467cd1fbc8e0727dbcf5e0c1b6dcd271.jpg  
  inflating: football players.v2i.coco/test/yt-NfzkB0v31Pg-0051_jpg.rf.50556582ff7415

In [18]:
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torchvision.transforms as T
from detr.models.backbone import build_backbone
from detr.models.transformer import build_transformer
from detr.models.detr import DETR
from detr.util.misc import nested_tensor_from_tensor_list
from detr.models.matcher import HungarianMatcher
from detr.models.detr import SetCriterion
from detr.datasets.coco import CocoDetection
from types import SimpleNamespace
from detr.datasets import transforms as T_detr



SHORT_TRAIN_CFG = {
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "epochs": 5,
    "batch_size": 4,
    "num_classes": 2,
    "num_queries": 100,

    # Required for DETR backbone/transformer builders
    "hidden_dim": 256,
    "position_embedding": "sine",
    "backbone": "resnet50",
    "dilation": False,
    "enc_layers": 6,
    "dec_layers": 6,
    "dim_feedforward": 2048,
    "dropout": 0.1,
    "nheads": 8,
    "pre_norm": False,
    "return_interm_layers": False,
    "lr_backbone": 1e-5,       # Learning rate for the backbone
    "masks": False
}


# Data Transform
def make_transforms():
    return T_detr.Compose([
        T_detr.RandomResize([400, 500, 600]),
        T_detr.ToTensor(),
        T_detr.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

# Collate Function
def collate_fn(batch):
    return tuple(zip(*batch))

# Custom Lightning Module
class DETRLightning(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg

        # Build the model
        args = SimpleNamespace(**cfg)
        backbone = build_backbone(args)
        transformer = build_transformer(args)


        self.model = DETR(
            backbone=backbone,
            transformer=transformer,
            num_classes=cfg["num_classes"],
            num_queries=cfg["num_queries"],
            aux_loss=True  # Set True if your loss includes auxiliary outputs
        )

        # Loss setup
        matcher = HungarianMatcher()
        weight_dict = {'loss_ce': 1, 'loss_bbox': 5, 'loss_giou': 2}
        losses = ['labels', 'boxes', 'cardinality']
        self.criterion = SetCriterion(
            cfg["num_classes"],
            matcher=matcher,
            weight_dict=weight_dict,
            eos_coef=0.1,
            losses=losses
        )

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        images = list(image.to(self.device) for image in images)
        targets = [{k: v.to(self.device) for k, v in t.items()} for t in targets]

        outputs = self(images)
        loss_dict = self.criterion(outputs, targets)
        loss = sum(loss_dict[k] for k in loss_dict if k in self.criterion.weight_dict)

        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, targets = batch
        images = list(image.to(self.device) for image in images)
        targets = [{k: v.to(self.device) for k, v in t.items()} for t in targets]

        outputs = self(images)
        loss_dict = self.criterion(outputs, targets)
        loss = sum(loss_dict[k] for k in loss_dict if k in self.criterion.weight_dict)

        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(
            self.parameters(),
            lr=self.cfg["lr"],
            weight_decay=self.cfg["weight_decay"]
        )

# Load Dataset
train_ds = CocoDetection(
    "/content/football players.v2i.coco/train",
    "/content/football players.v2i.coco/_trainannotations.coco.json",
    transforms=make_transforms(),
    return_masks=False
)

val_ds = CocoDetection(
    "/content/football players.v2i.coco/valid",
    "/content/football players.v2i.coco/_validannotations.coco.json",
    transforms=make_transforms(),
    return_masks=False
)

train_loader = DataLoader(train_ds, batch_size=SHORT_TRAIN_CFG["batch_size"], shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_ds, batch_size=SHORT_TRAIN_CFG["batch_size"], collate_fn=collate_fn)

# Training
trainer = pl.Trainer(
    max_epochs=SHORT_TRAIN_CFG["epochs"],
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    logger=pl.loggers.TensorBoardLogger("lightning_logs"),
    enable_checkpointing=True
)

model = DETRLightning(SHORT_TRAIN_CFG)
trainer.fit(model, train_loader, val_loader)


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | model     | DETR         | 41.5 M | train
1 | criterion | SetCriterion | 0      | train
---------------------------------------------------
41.3 M    Trainable params
222 K     Non-trainable params
41.5 M    Total params
166.008   Total estimated model params size (MB)
313       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
class DETRLightning(pl.LightningModule):
    def __init__(self, args):
        super().__init__()
        self.model = build_detr(args)
        self.criterion = SetCriterion(...)
        # other init code...

    def validation_step(self, batch, batch_idx):
        samples, targets = batch
        outputs = self.model(samples)
        loss_dict = self.criterion(outputs, targets)
        total_loss = sum(loss_dict[k] for k in loss_dict if k != 'cardinality')
        self.log("val_loss", total_loss)
        return total_loss


In [ ]:
trainer.validate(model, dataloaders=val_loader)

In [ ]:
torch.save(model.model, "detr_model.pth")